In [ ]:
%load_ext autoreload
%autoreload 2

2 agents:

Multiagent:
Collide: 0.0 0.0
Unsafe: 0.0 0.0

Singleagent:
Collide: 1.5 2.9410882339705484
Unsafe: 0.0 0.0

3 agents:
Multiagent:
0.0 0.0
2.3 0.7810249675906654

Singleagent:
1.6 2.973213749463701
2.0 0.6324555320336759

4 agents:
Multiagent
0.1 0.30000000000000004
1.0 0.6324555320336759

Singleagent
1.9 2.6248809496813377
0.5 0.6708203932499369

5 agents:
Multiagent
0.7 0.9
47.6 0.9165151389911679

Singleagent
3.0 2.898275349237888
46.8 0.4

6 agents:
Multiagent
1.4 1.0198039027185568
28.0 2.8635642126552705

Singleagent
6.5 3.0740852297878796
26.5 1.91049731745428

7 agents:
Multiagent:
1.5 1.3601470508735443
11.5 2.5787593916455256

Singleagent:
6.8 1.6613247725836149
8.3 1.3453624047073711

In [ ]:
from __future__ import division, print_function, absolute_import

import time

import GPy
import matplotlib.pyplot as plt
import numpy as np

from multiagent_grid_world import (compute_true_safe_set, compute_S_hat0,
                                compute_true_S_hat, draw_gp_sample, MultiagentGridWorldAgent)

In [ ]:
num_agent = 2
agent_explore_kernels = []
agent_explore_liks = []
# agent_exploit_kernels = []
# agent_exploit_liks = []
noise = 0.001

for agent in range(num_agent):
    agent_explore_kernels += [GPy.kern.RBF(input_dim=4, lengthscale=(2., 2., 2., 2.), ARD=True)]
    agent_explore_liks += [GPy.likelihoods.Gaussian(variance=noise ** 2)]
    agent_explore_liks[agent].constrain_bounded(1e-6, 1)
    
# for agent in range(num_agent):
#     agent_exploit_kernels += [GPy.kern.RBF(input_dim=4, lengthscale=(2., 2., 2., 2.), ARD=True)]
#     agent_exploit_liks += [GPy.likelihoods.Gaussian(variance=noise ** 2)]
#     agent_exploit_liks[agent].constrain_bounded(1e-6, 1)

In [4]:
agent_rewards_kernels = []
agent_rewards_liks = []
noise = 0.001

for agent in range(num_agent):
    agent_rewards_kernels += [GPy.kern.RBF(input_dim=2, lengthscale=(2., 2.), variance=1., ARD=True)]
    agent_rewards_liks += [GPy.likelihoods.Gaussian(variance=noise ** 2)]
    agent_rewards_liks[agent]

In [5]:
# Define world
world_shape = (5, 5)
step_size = (0.5, 0.5)

In [6]:
# Define coordinates
n, m = world_shape
step1, step2 = step_size
xx, yy = np.meshgrid(np.linspace(0, (n - 1) * step1, n),
                     np.linspace(0, (m - 1) * step2, m),
                     indexing="ij")
coord = np.vstack((xx.flatten(), yy.flatten())).T

In [7]:
# Safety threhsold
h = -0.25

# Lipschitz
L = 0

# Scaling factor for confidence interval
beta = 2

collide_threshold = 0.99

In [9]:
import copy
# Data to initialize GP
agent_gps = []
agent_rewards_gps = []
altitude_kernel = GPy.kern.RBF(input_dim=2, lengthscale=(2., 2.), variance=1., ARD=True)
altitude_lik = GPy.likelihoods.Gaussian(variance=noise ** 2)
altitude_lik.constrain_bounded(1e-6, 10000.)
altitudes, coord = draw_gp_sample(altitude_kernel, world_shape, step_size)
print(altitudes)

for agent in range(num_agent):
    noise = 0.001
    n_samples = 1
    ind = np.random.choice(range(altitudes.size), n_samples)
    X = coord[ind, :]
    Y = altitudes[ind].reshape(n_samples, 1) + np.random.randn(n_samples, 1)
    agent_gps += [GPy.core.GP(X, Y, copy.deepcopy(altitude_kernel), copy.deepcopy(altitude_lik))]
    agent_rewards_gps += [GPy.core.GP(X, Y, copy.deepcopy(altitude_kernel), copy.deepcopy(altitude_lik))]

reconstraining parameters Gaussian_noise


[ 0.02440064 -0.19850443 -0.55144736 -0.99568087 -1.46362983  0.43057205
  0.25993253 -0.04180778 -0.4524123  -0.92367692  0.74268053  0.63671968
  0.40978713  0.06478553 -0.37616521  0.80686004  0.77718224  0.65093172
  0.4119865   0.05280927  0.57692184  0.6357917   0.63521296  0.54122631
  0.31646203]


In [10]:
import numpy as np

n_samples = 1
agent_explore_gps = []
# agent_exploit_gps = []
X = []
Y = []

for agent in range(num_agent):
    for i in range(world_shape[0]):
        for j in range(world_shape[1]):
            for i_ in range(world_shape[0]):
                for j_ in range(world_shape[1]):
                    X += [[i * step_size[0], j * step_size[1], i_ * step_size[0], j_ * step_size[1]]]
                    Y += [[np.random.uniform(-1,0,1000)[0]]]

    X = np.array(X)
    Y = np.array(Y)
    mean_func_explore = GPy.core.Mapping(4, 1, name='agent_explore')
    mean_func_explore.f = lambda x: 0.5
    mean_func_explore.update_gradients = lambda a,b: 0
    mean_func_explore.gradients_X = lambda a,b: 0
    agent_explore_gps += [GPy.core.GP(X, Y, agent_explore_kernels[agent], agent_explore_liks[agent], mean_function=mean_func_explore)]
    
#     mean_func_exploit = GPy.core.Mapping(4, 1, name='agent_exploit')
#     mean_func_exploit.f = lambda x: 0.5
#     mean_func_exploit.update_gradients = lambda a,b: 0
#     mean_func_exploit.gradients_X = lambda a,b: 0
#     agent_exploit_gps += [GPy.core.GP(X, Y, agent_exploit_kernels[agent], agent_exploit_liks[agent], mean_function=mean_func_exploit)]
    

In [12]:
import copy
from tqdm import tqdm
collides = []
unsafes = []

for epoch in range(10):
    print(epoch)
    count_collide = 0
    count_unsafe = 0
    agents = []
    agent_S0s = []
    agent_S_hat0s = []
    agent_pos = []
    while len(agent_pos) != num_agent:
        agent_pos = []
        for agent in range(num_agent):
            S0 = np.zeros((np.prod(world_shape), 5), dtype=bool)
            S0[:, 0] = True
            S_hat0 = compute_S_hat0(np.nan, world_shape, 4, altitudes,
                                    step_size, h)
            for i in range(len(np.where(S_hat0[:, 1])[0])):
                if np.where(S_hat0[:, 1])[0][i] not in agent_pos:
                    agent_pos += [np.where(S_hat0[:, 1])[0][i]]
                    break
            agent_S0s += [S0]
            agent_S_hat0s += [S_hat0]

    for agent in range(num_agent):
        cur_agent_explore_gps = copy.deepcopy(agent_explore_gps)
        cur_agent_explore_gps.pop(agent)
        cur_agent_rewards_gps = copy.deepcopy(agent_rewards_gps)
        cur_agent_rewards_gps.pop(agent)
        cur_agent_pos = copy.deepcopy(agent_pos)
        cur_agent_pos.pop(agent)
        agents += [
            MultiagentGridWorldAgent(
                agent_gps[agent], 
                cur_agent_explore_gps,
                cur_agent_rewards_gps,
                world_shape, 
                step_size, 
                beta, 
                altitudes, 
                h, 
                collide_threshold, 
                agent_S0s[agent],
                agent_S_hat0s[agent], 
                agent_pos[agent], 
                L, 
                cur_agent_pos,
                [0.5 for _ in range(num_agent - 1)],
            )
        ]

#     for i in tqdm(range(10)):
#         agent_actions = [0 for agent in range(num_agent)]
#         agent_next_samples = []
#         agent_states = []
#         for agent in range(num_agent):
#             agents[agent].update_sets()
#             next_sample = agents[agent].target_sample()
#     #         print('agent:' + str(agent))
#     #         print(next_sample)
#             agent_actions[agent] = next_sample[1]
#             agent_next_samples += [next_sample]
#             agent_states += [next_sample[0]]
#             if altitudes[next_sample[0]] < h:
#     #             print('entered an unsafe state')
#                 count_unsafe += 1
#     #         print()

#     #     print(agent_states)
#         if len(set(agent_states)) < len(agent_states):
#             count_collide += 1
#     #     print()

#         for agent in range(num_agent):
#             agents[agent].add_observation(agent_next_samples[agent][0], agent_next_samples[agent][1], agent_actions)
            
#     collides += [count_collide]
#     unsafes += [count_unsafe]

print(collides)
print(unsafes)
print(np.mean(collides), np.std(collides))
print(np.mean(unsafes), np.std(unsafes))

0
Edges Added


TypeError: can only concatenate tuple (not "float") to tuple

In [ ]:
import copy
from tqdm import tqdm
from singleagent_grid_world import SingleagentGridWorldAgent
collides = []
unsafes = []

for epoch in range(10):
    print(epoch)
    count_collide = 0
    count_unsafe = 0
    agents = []
    agent_S0s = []
    agent_S_hat0s = []
    agent_pos = []
    while len(agent_pos) != num_agent:
        agent_pos = []
        for agent in range(num_agent):
            S0 = np.zeros((np.prod(world_shape), 5), dtype=bool)
            S0[:, 0] = True
            S_hat0 = compute_S_hat0(np.nan, world_shape, 4, altitudes,
                                    step_size, h)
            for i in range(len(np.where(S_hat0[:, 1])[0])):
                if not np.where(S_hat0[:, 1])[0][i] in agent_pos:
                    agent_pos += [np.where(S_hat0[:, 1])[0][i]]
                    break
            agent_S0s += [S0]
            agent_S_hat0s += [S_hat0]

    for agent in range(num_agent):
        cur_agent_action_gps = copy.deepcopy(agent_action_gps)
        cur_agent_action_gps.pop(agent)
        cur_agent_pos = copy.deepcopy(agent_pos)
        cur_agent_pos.pop(agent)
        agents += [
            SingleagentGridWorldAgent(
                agent_gps[agent], 
                cur_agent_action_gps, 
                world_shape, 
                step_size, 
                beta, 
                altitudes, 
                h, 
                collide_threshold, 
                agent_S0s[agent],
                agent_S_hat0s[agent], 
                agent_pos[agent], 
                L, 
                cur_agent_pos
            )
        ]

    for i in tqdm(range(100)):
        agent_actions = [0 for agent in range(num_agent)]
        agent_next_samples = []
        agent_states = []
        for agent in range(num_agent):
            agents[agent].update_sets()
            next_sample = agents[agent].target_sample()
    #         print('agent:' + str(agent))
    #         print(next_sample)
            agent_actions[agent] = next_sample[1]
            agent_next_samples += [next_sample]
            agent_states += [next_sample[0]]
            if altitudes[next_sample[0]] < h:
    #             print('entered an unsafe state')
                count_unsafe += 1
    #         print()

    #     print(agent_states)
        if len(set(agent_states)) < len(agent_states):
            count_collide += 1
    #     print()

        for agent in range(num_agent):
            agents[agent].add_observation(agent_next_samples[agent][0], agent_next_samples[agent][1], agent_actions)
            
    collides += [count_collide]
    unsafes += [count_unsafe]
    
print(collides)
print(unsafes)
print(np.mean(collides), np.std(collides))
print(np.mean(unsafes), np.std(unsafes))